In [ ]:
#importing the necessary libraries
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import load_model
import random

In [ ]:
# Download NLTK data (wordnet, punkt)
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Load the trained model
model = load_model('chatbot_model.h5')

In [ ]:
# Load words and classes
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [ ]:
# Load the intents file
with open('intents.json') as file:
    intents = json.load(file)

In [ ]:
# Function to preprocess input
def clean_up_sentence(sentence):
    # Tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # Lemmatize each word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [ ]:
# Return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # Tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # Bag of words
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

In [ ]:
# Function to predict the class
def predict_class(sentence, model):
    # Filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    # Adjust the shape of the input to match the model's expected input shape
    p = np.expand_dims(p, axis=0)
    res = model.predict(p)[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

In [ ]:
    # Sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [ ]:
# Function to get the response
def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = get_response(ints, intents)
    return res

In [ ]:
# Interact with the chatbot
print("Chatbot is ready to talk! Type 'quit' to stop.")
while True:
    message = input("")
    if message.lower() == "quit":
        break
    response = chatbot_response(message)
    print(response)